In [1]:
import requests

In [2]:
client_id = '626'
client_secret = '3fa520ce21bab7c1d81a15c9d0676e36'

In [16]:
credentials_endpt = 'https://booking.library.gwu.edu/1.1/oauth/token'
calendars_endpt = 'https://booking.library.gwu.edu/1.1/calendars'
bookings_endpt = 'https://booking.library.gwu.edu/1.1/space/bookings'

In [4]:
def create_cred_body(client_id: str, client_secret: str):
    '''Creates the body of a POST request for obtaining an access token, given a client secret and credentials.'''
    return {'client_id': client_id,
           'client_secret': client_secret,
           'grant_type': 'client_credentials'}

In [5]:
def get_access_token(cred_body: dict):
    '''Retrieves an access token from the LibCal API, given a request body as specified.'''
    try:
        resp = requests.post(credentials_endpt, json=cred_body)
        resp.raise_for_status()
        token_data = resp.json()
        if 'error' in token_data:
            raise Exception(token_data)
            #return resp
        return token_data
    except Exception as e:
        print(e)
        raise

In [6]:
cred_body = create_cred_body(client_id, client_secret)
token_data = get_access_token(cred_body)

In [7]:
token_data

{'access_token': 'f3822cfa5ca7920fe2ad4f1ecd5120f5a667fb34',
 'expires_in': 3600,
 'token_type': 'Bearer',
 'scope': 'cal_r ev_r ms_r rm_r eq_r sp_r h_r'}

In [8]:
def create_header(token_data: dict):
    '''Creates an authorization header with the supplied token from the credentials API.'''
    try:
        token = token_data['access_token']
        return {'Authorization': f'Bearer {token}'}
    except KeyError:
        print('Access token missing from token data.')
        raise

In [9]:
def get_calendars(header: dict):
    '''Retrieves the calendars available in the system, for querying events from a specific calendar.
    Header argument should contain the authorization token.'''
    try:
        resp = requests.get(calendars_endpt, headers=header)
        resp.raise_for_status()
        cal_data = resp.json()
        if 'error' in cal_data:
            raise Exception(cal_data)
        return cal_data
    except Exception as e:
        print('Error getting calendars.')
        raise
    

In [10]:
header = create_header(token_data)
calendars = get_calendars(header)

In [21]:
def create_booking_params(date: str = None):
    '''Sets querystr parameters for the space/bookings endpoint. 
    Date should be in the format YYYY-MM-DD.'''
    params = {'formAnswers': 1,
             'limit': 100} # Max value
    if date:
        params['date'] = date
    return params
        

In [27]:
def get_bookings(header: dict, booking_id: str = None, date: str = None):
    '''Retrieves bookings from LibCal.'''
    if booking_id:
        url = f'{bookings_endpt}/{booking_id}'
    else:
        url = bookings_endpt
    try:
        params = create_booking_params(date)
        resp = requests.get(url, headers=header, params=params)
        resp.raise_for_status()
        bookings_data = resp.json()
        if 'error' in bookings_data:
            raise Exception(bookings_data)
        return bookings_data
    except Exception as e:
        print('Error getting bookings')
        raise

I think the `space/bookings` endpoint is the one to use for the room reservation system. Note that it seems possible to retrieve appointments only for one day at a time. By default, the date is today's date. Another date can be supplied as a querystring parameter.

In [31]:
get_bookings(header)

[{'bookId': 'cs_jPvGgncq',
  'eid': 68247,
  'cid': 19869,
  'lid': 8827,
  'fromDate': '2020-08-07T11:00:00-04:00',
  'toDate': '2020-08-07T13:00:00-04:00',
  'firstName': 'Laura',
  'lastName': 'Wrubel',
  'email': 'lwrubel@gwu.edu',
  'status': 'Confirmed'}]

In [32]:
get_bookings(header, date='2020-08-11')

[{'bookId': 'cs_mO5Gb5f8',
  'eid': 75467,
  'cid': 19872,
  'lid': 8827,
  'fromDate': '2020-08-11T09:00:00-04:00',
  'toDate': '2020-08-11T11:00:00-04:00',
  'firstName': 'Laura',
  'lastName': 'Wrubel',
  'email': 'lwrubel@gwu.edu',
  'status': 'Confirmed'}]